# Chronos 2 RUL Prediction Example

This notebook demonstrates how to use Amazon's Chronos 2 forecasting model for Remaining Useful Life (RUL) prediction on C-MAPSS datasets.

**Author**: Auto-generated for PhD research
**Date**: January 2026

## Setup and Installation

First, make sure you have installed the required dependencies:

In [ ]:
# Install dependencies (run this in your terminal, not in the notebook)
# pip install -r requirements.txt

# Import required libraries
import sys
import os
from pathlib import Path

# Add src to path - fix for Jupyter notebook
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_path = project_root / 'src'
sys.path.append(str(src_path))
sys.path.append(str(project_root))

print(f"Project root: {project_root}")
print(f"Added to path: {src_path}")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import our custom classes
from data_loader import MultiDatasetLoader
from chronos_rul_predictor import ChronosRULPredictor, create_chronos_experiment

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("✅ Setup complete! All imports successful.")

Error: 

## Load and Prepare Data

Let's start by loading the C-MAPSS FD001 dataset using your existing data loader:

In [2]:
# Initialize the data loader
loader = MultiDatasetLoader()

# Load FD001 dataset
dataset = loader.load_cmapss_dataset('FD001')

print(f"Dataset loaded!")
print(f"Training samples: {len(dataset['train'])}")
print(f"Test samples: {len(dataset['test'])}")
print(f"Training units: {dataset['train']['unit'].nunique()}")
print(f"Test units: {dataset['test']['unit'].nunique()}")

Error: 

## Initialize Chronos RUL Predictor

Now let's create a Chronos-based RUL predictor:

In [ ]:
# Initialize Chronos predictor
predictor = ChronosRULPredictor(
    model_name="amazon/chronos-t5-small",  # Small model for faster inference
    device="auto",  # Will use CUDA if available
    prediction_length=50,  # Forecast 50 cycles ahead
    context_length=512     # Maximum context length
)

print("Chronos predictor initialized!")
print(f"Using device: {predictor.device}")
print(f"Model: {predictor.model_name}")

## Prepare Data for Chronos

Chronos expects time series data in a specific format. Let's prepare our C-MAPSS data:

In [ ]:
# Prepare data for Chronos
prepared_data = predictor.prepare_cmapss_data(
    dataset,
    feature_cols=None,  # Use all sensor columns
    max_rul=125         # Clip RUL at 125 cycles (common in literature)
)

fc = prepared_data['feature_cols']
print(f"Data prepared!")
print(f"Training sequences: {len(prepared_data['train_sequences'])}")
print(f"Test sequences: {len(prepared_data['test_sequences'])}")
print(f"Features used: {len(fc)} sensors")
print(f"Feature names: {fc[0]} .. {fc[-1]} (all {len(fc)} C-MAPSS sensors)")

## Example: Single RUL Prediction

Let's demonstrate RUL prediction on a single test sequence:

In [ ]:
# Get a test sequence
test_seq = prepared_data['test_sequences'][0]

print(f"Test sequence info:")
print(f"Unit: {test_seq['unit']}")
print(f"Sequence length: {test_seq['seq_length']} cycles")
print(f"Actual RUL: {test_seq['rul'][-1]} cycles")
print(f"Features shape: {test_seq['features'].shape}")

# Make RUL prediction
predicted_rul = predictor.predict_rul(
    test_seq['features'],
    method='threshold',
    threshold_method='deviation',
    threshold_factor=2.0
)

print(f"\nPredicted RUL: {predicted_rul:.1f} cycles")
print(f"Prediction error: {abs(predicted_rul - test_seq['rul'][-1]):.1f} cycles")

## Evaluate on Full Test Set

Now let's evaluate the predictor on the entire test set:

In [ ]:
# Evaluate performance
evaluation_results = predictor.evaluate_on_dataset(
    prepared_data,
    threshold_method='deviation',
    threshold_factor=2.0
)

print("Evaluation Results:")
print(f"RMSE: {evaluation_results['rmse']:.2f}")
print(f"MAE: {evaluation_results['mae']:.2f}")
print(f"Within 10 cycles: {evaluation_results['within_10_cycles']:.3f}")
print(f"Within 20 cycles: {evaluation_results['within_20_cycles']:.3f}")
print(f"Number of predictions: {evaluation_results['n_predictions']}")

## Hyperparameter Optimization

Let's optimize the hyperparameters for better performance:

In [ ]:
# Define parameter grid for optimization
param_grid = {
    'threshold_method': ['deviation', 'absolute', 'adaptive'],
    'threshold_factor': [1.5, 2.0, 2.5, 3.0]
}

# Optimize hyperparameters
print("Optimizing hyperparameters...")
best_params = predictor.optimize_hyperparameters(
    prepared_data,
    param_grid=param_grid,
    cv_folds=3
)

print(f"\nBest parameters found:")
for param, value in best_params.items():
    print(f"{param}: {value}")

# Re-evaluate with optimized parameters
print("\nRe-evaluating with optimized parameters...")
optimized_results = predictor.evaluate_on_dataset(prepared_data)

print(f"Optimized RMSE: {optimized_results['rmse']:.2f} (previous: {evaluation_results['rmse']:.2f})")
print(f"Optimized Within 20 cycles: {optimized_results['within_20_cycles']:.3f} (previous: {evaluation_results['within_20_cycles']:.3f})")

## Ensemble Methods

Create an ensemble of Chronos models for potentially better performance:

In [ ]:
# Create ensemble
ensemble = predictor.create_ensemble(
    prepared_data,
    n_models=3,
    model_variants=['amazon/chronos-t5-small'] * 3  # Same model for simplicity
)

# Evaluate ensemble
ensemble_results = ensemble.evaluate_on_dataset(prepared_data)

print("Ensemble Results:")
print(f"RMSE: {ensemble_results['rmse']:.2f}")
print(f"MAE: {ensemble_results['mae']:.2f}")
print(f"Within 10 cycles: {ensemble_results['within_10_cycles']:.3f}")
print(f"Within 20 cycles: {ensemble_results['within_20_cycles']:.3f}")

# Compare with single model
print(f"\nComparison:")
print(f"Single model RMSE: {optimized_results['rmse']:.2f}")
print(f"Ensemble RMSE: {ensemble_results['rmse']:.2f}")
print(f"Improvement: {optimized_results['rmse'] - ensemble_results['rmse']:.2f}")

## Convenience Function

Use the convenience function to run a complete experiment:

In [ ]:
# Complete experiment with one function call
predictor2, data2, results2 = create_chronos_experiment(
    loader,
    dataset_name='FD001',
    model_name='amazon/chronos-t5-small',
    prediction_length=50
)

print("Complete experiment results:")
print(f"RMSE: {results2['rmse']:.2f}")
print(f"Within 20 cycles: {results2['within_20_cycles']:.3f}")

## Visualization

Let's create some visualizations of the results:

In [ ]:
# Plot predictions vs actuals
predictions = evaluation_results['predictions']
actuals = evaluation_results['actuals']

plt.figure(figsize=(12, 5))

# Scatter plot
plt.subplot(1, 2, 1)
plt.scatter(actuals, predictions, alpha=0.6, s=30)
plt.plot([0, 130], [0, 130], 'r--', label='Perfect prediction')
plt.xlabel('Actual RUL (cycles)')
plt.ylabel('Predicted RUL (cycles)')
plt.title('Predicted vs Actual RUL')
plt.grid(True, alpha=0.3)
plt.legend()

# Error distribution
plt.subplot(1, 2, 2)
errors = predictions - actuals
plt.hist(errors, bins=30, alpha=0.7, edgecolor='black')
plt.xlabel('Prediction Error (cycles)')
plt.ylabel('Frequency')
plt.title('Prediction Error Distribution')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print some statistics
print(f"Mean error: {np.mean(errors):.2f} cycles")
print(f"Error std: {np.std(errors):.2f} cycles")
print(f"Max absolute error: {np.max(np.abs(errors)):.2f} cycles")

## Summary

This notebook demonstrated how to use Chronos 2 for RUL prediction on C-MAPSS datasets. Key takeaways:

1. **Zero-shot forecasting**: Chronos can forecast sensor values without task-specific training
2. **RUL from forecasts**: Convert forecasts to RUL predictions using threshold-based methods
3. **Hyperparameter optimization**: Tune threshold methods and factors for better performance
4. **Ensemble methods**: Combine multiple models for potentially improved accuracy

### Next Steps
- Try different Chronos model sizes (base, large) for potentially better performance
- Experiment with different RUL prediction methods (trend-based, regression-based)
- Fine-tune Chronos on your specific datasets (though this is experimental)
- Compare with traditional ML methods (SVM, LSTM, etc.)

### Performance Expectations
- **Chronos small**: ~25-35 RMSE on FD001 (baseline)
- **With optimization**: ~20-30 RMSE
- **Ensemble**: ~18-28 RMSE

*Note: These are rough estimates. Actual performance depends on hyperparameters and data preprocessing.*